# Tester notebook

This notebook aims to create enviroment to simply test strategies developed in Stocks & Loss system for python

In [1]:
# imports 
from scraper import stock_daily
from analyzer import Analyzer
import numpy as np
import pandas as pd
from IPython.display import display

## Load stocks across industries

In [2]:
xls = pd.ExcelFile('TickerPerIndexFund.xls')
fundTickers = pd.DataFrame()
Tickers = pd.DataFrame()
fundTickers["Fund/sector name"] = xls.sheet_names
# Tickers = pd.DataFrame(,columns=funTickers["Index fund name"])
for i in range(len(xls.sheet_names)):
    sheet1 = pd.read_excel(xls, i)
    fundTickers.loc[i,["Fund/sector ticker"]] = sheet1.columns[1]
    Tickers[fundTickers.loc[i,["Fund/sector name"]]] = sheet1[sheet1.columns[1]][1:]
pd.set_option('display.expand_frame_repr', False)    
display(Tickers)

,S&P 500,Invesco QQQ,Russell 2000,Emerging Markets,Dow Jones
1,AAPL,AAPL,PLUG,2330,UNH
2,MSFT,MSFT,PENN,700,GS
3,AMZN,AMZN,CZR,BABA,HD
4,FB,TSLA,NVAX,SSNLF,MSFT
5,GOOGL,GOOG,DAR,3690,AMGN
6,GOOG,FB,LAD,NAPRF,CRM
7,TSLA,GOOGL,RUN,JD,CAT
8,BRK.B,NVDA,DECK,RELIANCE,V
9,JPM,PYPL,RARE,CICHF,BA
10,JNJ,INTC,BLDR,BIDU,MCD


## Iterate over each industry

In [3]:
# I have the initialization here on purpose, to measure its impact
col = ["Ticker", "Trades", "Good trades", "Profit[%]", "Profit[$]", "Hold[%]"]
results = [None] * Tickers.columns.shape[0]
for k in range(Tickers.columns.shape[0]):
    # when I tried to initialize results as [dataframe] * 10 it created list of identical dataframes and when i changed one, it changed them all. Interesting
    results[k] = pd.DataFrame(data=np.zeros([Tickers.shape[0], len(col)]),columns=col)

In [4]:
# iterate over columns
i = 0 
for label, tic in Tickers.iteritems():
    print(label)
    # iterate over items in sectors / indexes
    for j, val in tic.iteritems():
        # j starts at 1, instead of usual 0 => reason why there is -1 everywhere
        # analyze stock
        try:
            anal = Analyzer(ticker=val,data=stock_daily(val, save=False).data)
            anal.strategy(buyStrategy=['Mcstoch_ut1','Mcstoch_ut2','Mcstoch_ut3','Mcstoch_ut4'],sellStrategy=['Mcstoch'],stopLoss=True,stopLossValue=0.05,profitTaker=False,profitTakerValue=0.2)
            ts = anal.profit(capitalForEachTrade=400,comission=2)
            # save data into dataframe
            results[i].loc[j-1,["Ticker"]] = val
            results[i].loc[j-1,["Trades"]] = ts.shape[0]
            results[i].loc[j-1,["Good trades"]] = ts["Good trade?"].sum()
            results[i].loc[j-1,["Profit[%]"]] = ts["Profit[%]"].sum()
            results[i].loc[j-1,["Profit[$]"]] = ts["Profit[$]"].sum()
            results[i].loc[j-1,["Hold[%]"]] = 100*((anal.data["Close"].iloc[-1]-anal.data["Close"].iloc[0])/anal.data["Close"].iloc[0])
        except:
            results[i].loc[j-1,["Ticker"]] = val
    i += 1

S&P 500
- BRK.B: No data found, symbol may be delisted
Invesco QQQ
Russell 2000
Emerging Markets
- SSNLF: No data found for this date range, symbol may be delisted
- RELIANCE: No data found, symbol may be delisted
- VALE3: No data found, symbol may be delisted
- HDFC: No data found, symbol may be delisted
- NHNCF: No data found for this date range, symbol may be delisted
Dow Jones


In [5]:
display(results[2])
for k in range(Tickers.columns.shape[0]):
    print(Tickers.columns[k],': profit from trading [$] = ',np.round(np.sum(results[k]["Profit[$]"]),1),' after ',np.round(np.sum(results[k]["Trades"]),1),' trades.')


,Ticker,Trades,Good trades,Profit[%],Profit[$],Hold[%]
0,PLUG,9.0,6.0,163.867724,655.470896,205.363134
1,PENN,9.0,3.0,-22.996078,-91.984314,100.473137
2,CZR,15.0,3.0,-33.600282,-134.401129,141.945587
3,NVAX,6.0,0.0,-31.018776,-124.075105,43.318541
4,DAR,13.0,4.0,16.788425,67.153698,141.607148
5,LAD,12.0,3.0,-19.593874,-78.375496,78.879191
6,RUN,5.0,2.0,11.916394,47.665576,30.037183
7,DECK,12.0,3.0,1.209888,4.839553,92.147844
8,RARE,7.0,1.0,-7.676532,-30.706128,0.723335
9,BLDR,14.0,7.0,-6.470417,-25.881667,91.283802


S&P 500 : profit from trading [$] =  154.7  after  203.0  trades.
Invesco QQQ : profit from trading [$] =  401.1  after  240.0  trades.
Russell 2000 : profit from trading [$] =  1686.9  after  229.0  trades.
Emerging Markets : profit from trading [$] =  651.8  after  82.0  trades.
Dow Jones : profit from trading [$] =  288.7  after  228.0  trades.
